In [3]:
from transformers import GPT2LMHeadModel
from transformers import GPT2Tokenizer


tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_decoder = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer.pad_token = "<PAD>"


In [23]:
caption_length = 20 


caption = "Who is Modi "

tokens = tokenizer(
            caption,
            max_length=caption_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )


In [5]:
tokens

{'input_ids': tensor([[15496,   220, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256]]), 'attention_mask': tensor([[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}

In [24]:
import torch 

gpt_decoder.eval() 

with torch.no_grad():
        output_ids = gpt_decoder.generate(
            input_ids=tokens["input_ids"],
            max_length=30,
            num_beams=3,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    # Decode
tokenizer.decode(output_ids[0], skip_special_tokens=True)

'Who is Modi This is a rush transcript. Copy may not be'

In [22]:
import torch

def greedy_generate(model, tokenizer, input_ids, max_new_tokens=30, device="cpu"):
    model.eval()
    generated = input_ids.to(device)

    for _ in range(max_new_tokens):
        # Forward pass
        with torch.no_grad():
            outputs = model(input_ids=generated)
            logits = outputs.logits  # (B, T, V)
        
        # Get last token logits
        next_token_logits = logits[:, -1, :]  # (B, V)

        # Greedy: pick the highest-prob token
        next_token = torch.argmax(next_token_logits, dim=-1).unsqueeze(-1)  # (B, 1)

        # Append to sequence
        generated = torch.cat([generated, next_token], dim=1)

        # Stop if EOS generated
        if tokenizer.eos_token_id is not None and (next_token == tokenizer.eos_token_id).all():
            break

    return generated

# Example usage
tokens = tokenizer("Who is Modi ", return_tensors="pt")
output_ids = greedy_generate(gpt_decoder, tokenizer, tokens["input_ids"], max_new_tokens=30)
print(tokenizer.decode(output_ids[0], skip_special_tokens=True))


Who is Modi ?"

"He is the Prime Minister of India. He is the Prime Minister of India. He is the Prime Minister of India. He is
